## Minimisation of TTC7B structure

PI4KA-TCC7B was downloaded from the PDB (`PDB:6BQ1`), while its density map was downloaded from EMD (`EMD-7129`).

In [ ]:
### Loading pose
template = 'pdb6bq1.pdb'
edmap = 'emd_7129.map'

import pyrosetta

pyrosetta.init(extra_options='-no_optH false -mute all -ignore_unrecognized_res true -load_PDB_components false')

pose = pyrosetta.rosetta.core.pose.Pose()
params_paths = pyrosetta.rosetta.utility.vector1_string()
params_paths.extend(['E4S.params'])
resiset = pyrosetta.generate_nonstandard_residue_set(pose, params_paths)  # 
pyrosetta.rosetta.core.import_pose.pose_from_file(pose,
                                                  template)  # pose_from_pdbstring or pose_from_file  

In [ ]:
### Minimisation
scorefxnED = pyrosetta.get_fa_scorefxn()
ED = pyrosetta.rosetta.core.scoring.electron_density.getDensityMap(edmap)
print(ED.matchPose(pose))
sdsm = pyrosetta.rosetta.protocols.electron_density.SetupForDensityScoringMover()
sdsm.apply(pose)
elec_dens_fast = pyrosetta.rosetta.core.scoring.ScoreType.elec_dens_fast
for w in (30, 20, 10):
    scorefxnED.set_weight(elec_dens_fast, w)
    relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxnED, 5)
    relax.apply(pose)
reg_scorefxn = pyrosetta.get_fa_scorefxn()
print('How good is the density match:', ED.matchPose(pose))
print(reg_scorefxn.get_name(), reg_scorefxn(pose))
pose.dump_pdb(template.replace('.pdb','.relax.pdb'))

## Minimisation of TTC7A model
Model was aligned in PyMOL and disordered loop deleted.

In [ ]:
### Loading pose
template = 'wTTC7A.SwMod.delLoop.pdb'
edmap = 'emd_7129.map'

import pyrosetta

pyrosetta.init(extra_options='-no_optH false -mute all -ignore_unrecognized_res true -load_PDB_components false')

pose = pyrosetta.rosetta.core.pose.Pose()
params_paths = pyrosetta.rosetta.utility.vector1_string()
params_paths.extend(['E4S.params'])
resiset = pyrosetta.generate_nonstandard_residue_set(pose, params_paths)  # 
pyrosetta.rosetta.core.import_pose.pose_from_file(pose,
                                                  template)  # pose_from_pdbstring or pose_from_file

To prevent the model from exploding a first fixed backbone relax was done.

In [ ]:
### Minimisation fixed bb
movemap = pyrosetta.MoveMap()
pose2pdb = pose.pdb_info().pose2pdb
for i in range(1, pose.total_residue()+1):
    resi, chain = map(lambda x: int(x) if x.isdigit() else x, pose2pdb(i).split())
    if chain == 'B':
        movemap.set_bb(i, False)
        movemap.set_chi(i, True)
    else:
        movemap.set_bb(i, False)
        movemap.set_chi(i, False)


scorefxnED = pyrosetta.get_fa_scorefxn()
ED = pyrosetta.rosetta.core.scoring.electron_density.getDensityMap('emd_7129.map')
print(ED.matchPose(pose))
sdsm = pyrosetta.rosetta.protocols.electron_density.SetupForDensityScoringMover()
sdsm.apply(pose)
elec_dens_fast = pyrosetta.rosetta.core.scoring.ScoreType.elec_dens_fast
w = 30
scorefxnED.set_weight(elec_dens_fast, w)
relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxnED, 5)
relax.set_movemap(movemap)
relax.set_movemap_disables_packing_of_fixed_chi_positions(True)
relax.apply(pose)
reg_scorefxn = pyrosetta.get_fa_scorefxn()
print('How good is the density match:', ED.matchPose(pose))
print(reg_scorefxn.get_name(), reg_scorefxn(pose))

PyMOL has a nicer neighbour matching algorithm

In [ ]:
import pymol2

with pymol2.PyMOL() as pymol:
    pymol.cmd.load(TTC7B_template, 'TTC7B')
    resisB = [int(at.resi) for at in pymol.cmd.get_model('chain A and name CA and byres chain B around 5').atom]
    pymol.cmd.delete()
    pymol.cmd.load(TTC7A_template, 'TTC7A')
    resisA = [int(at.resi) for at in pymol.cmd.get_model('chain A and name CA and byres chain B around 5').atom]
    resis = list(set(resisA) + set(resisB))
    
movemap = pyrosetta.MoveMap()
pose2pdb = pose.pdb_info().pose2pdb
for i in range(1, pose.total_residue()+1):
    resi, chain = map(lambda x: int(x) if x.isdigit() else x, pose2pdb(i).split())
    if chain == 'B':
        movemap.set_bb(i, True)
        movemap.set_chi(i, True)
    elif chain == 'A' and resi in resis:
        movemap.set_bb(i, False)
        movemap.set_chi(i, True)
    else:
        movemap.set_bb(i, False)
        movemap.set_chi(i, False)


scorefxnED = pyrosetta.get_fa_scorefxn()
ED = pyrosetta.rosetta.core.scoring.electron_density.getDensityMap('emd_7129.map')
print(ED.matchPose(pose))
sdsm = pyrosetta.rosetta.protocols.electron_density.SetupForDensityScoringMover()
sdsm.apply(pose)
elec_dens_fast = pyrosetta.rosetta.core.scoring.ScoreType.elec_dens_fast
for w in (30, 20, 10, 1):
    scorefxnED.set_weight(elec_dens_fast, w)
    relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxnED, 5)
    relax.set_movemap(movemap)
    relax.set_movemap_disables_packing_of_fixed_chi_positions(True)
    relax.apply(pose)
reg_scorefxn = pyrosetta.get_fa_scorefxn()
print('How good is the density match:', ED.matchPose(pose))
print(reg_scorefxn.get_name(), reg_scorefxn(pose))
pose.dump_pdb('wTTC7A.SwMod.relax.pdb')